### Advanced RAG


In [1]:
!pip install -q dotenv jsonlines openai langchain langchain-openai langchain-community chromadb langchain-chroma tiktoken rank_bm25 pymupdf kiwipiepy

### 1. Indexing
데이터를 불러옵니다.

In [4]:
import os
from dotenv import load_dotenv
from langchain.schema import Document
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma
from langchain_community.document_loaders import PyMuPDFLoader

load_dotenv(override=True)

# 모든 PDF 파일을 glob으로 찾음
pdf_files = ["./data/langchain-rag-advanced-[삼성전자]분기보고서(2024.05.16).pdf", "./data/langchain-rag-advanced-[삼성전자]분기보고서(2024.11.14).pdf"]

# 각 PDF 파일에서 페이지별로 내용을 불러와 하나로 합침
all_papers=[]

for i, path_paper in enumerate(pdf_files):
    loader = PyMuPDFLoader(path_paper)
    pages = loader.load()
    doc = Document(page_content='', metadata = {'index':i, 'source':pages[0].metadata['source']})
    for page in pages:
        doc.page_content += page.page_content +' '

    doc.page_content = doc.page_content.replace('\n', ' ')
    for _ in range(10):
        doc.page_content = doc.page_content.replace('  ', ' ')
        doc.page_content = doc.page_content.replace('..', '.')

    all_papers.append(doc)

print(len(all_papers))
all_papers[0].page_content[:1000]

2


'목 차 분 기 보 고 서.1 【 대표이사 등의 확인 】.2 I. 회사의 개요.3 1. 회사의 개요.3 2. 회사의 연혁.3 3. 자본금 변동사항.3 4. 주식의 총수 등.3 5. 정관에 관한 사항.3 II. 사업의 내용.4 1. 사업의 개요.4 2. 주요 제품 및 서비스.6 3. 원재료 및 생산설비.7 4. 매출 및 수주상황.14 5. 위험관리 및 파생거래.19 6. 주요계약 및 연구개발활동.25 7. 기타 참고사항.30 III. 재무에 관한 사항.50 1. 요약재무정보.50 2. 연결재무제표.53 2-1. 연결 재무상태표.53 2-2. 연결 손익계산서.55 2-3. 연결 포괄손익계산서.56 2-4. 연결 자본변동표.57 2-5. 연결 현금흐름표.58 3. 연결재무제표 주석.60 1. 일반적 사항 (연결).60 2. 중요한 회계처리방침 (연결) .70 3. 범주별 금융상품 (연결) .72 4. 공정가치금융자산 (연결).73 5. 재고자산 (연결) .75 6. 관계기업 및 공동기업 투자 (연결).75 7. 유형자산 (연결) .79 8. 무형자산 (연결) .81 9. 차입금 (연결).82 10. 사채 (연결) .83 11. 순확정급여부채(자산) (연결).84 12. 충당부채 (연결) .85 13. 우발부채와 약정사항 (연결) .87 14. 계약부채 (연결) .87 15. 자본금 (연결).88 16. 연결이익잉여금 (연결) .89 17. 기타자본항목 (연결).90 18. 비용의 성격별 분류 (연결) .91 19. 판매비와관리비 (연결) .92 20. 기타수익 및 기타비용 (연결).93 21. 금융수익 및 금융비용 (연결).93 22. 법인세비용 (연결).94 23. 주당이익 (연결) .95 24. 현금흐름표 (연결).95 25. 재무위험관리 (연결).98 26. 부문별 보고 (연결).110 27. 특수관계자와의 거래 (연결) .112 28. 매각예정분류(처분자산집단) (연결) .114 4. 재무제표.116 4-1. 재무상태표.116 4-2. 손익계산서.118 4-

In [5]:
import tiktoken

encoder = tiktoken.encoding_for_model('gpt-4o-mini')
for paper in all_papers:
    print(len(paper.page_content), len(encoder.encode(paper.page_content)), paper.metadata['source'])

189758 105094 ./data/langchain-rag-advanced-[삼성전자]분기보고서(2024.05.16).pdf
178571 100688 ./data/langchain-rag-advanced-[삼성전자]분기보고서(2024.11.14).pdf


### 2. Chunking

In [6]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
import tiktoken

token_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    # 토큰 수 기반의 청킹 (from_huggingface_tokenizer도 가능)
    model_name="gpt-4o-mini",
    chunk_size=1000,
    chunk_overlap=200,
)


token_chunks = token_splitter.split_documents(all_papers)
print(len(token_chunks))

260


In [7]:
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model = 'text-embedding-3-large', chunk_size = 100)

Chroma().delete_collection()


db = Chroma(
    embedding_function=embeddings,
    persist_directory="./chroma_Web",
    collection_metadata={'hnsw:space':'l2'}
)

db.add_documents(token_chunks)

# filter 옵션을 통해 특정 메타데이터를 가진 벡터만 검색 가능
# retriever = db.as_retriever(search_kwargs={"k": 5,"filter":{'author':'Hyungho Byun'}})
retriever = db.as_retriever(search_kwargs={"k": 5})